In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools
import networkx as nx
import itertools

import portion as P
import re
import datetime
import queue
import sympy as sym

# interval = P.empty()
# for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
#     interval = interval | P.closed(i, j)
# print(interval)

In [2]:
scipy.signal.correlate2d(np.array([
    [1,2,0],
    [1,0,0],
    [1,0,0]]), np.array([[1,1,1],
                         [1,1,0],
                         [1,0,0]]), mode='same')

array([[1, 4, 2],
       [4, 5, 2],
       [2, 2, 0]])

In [23]:
# new_positions = {2:'a',3:'a',1:'a',10:'b',23:'b'}

# flipped = {}
# for k, v in new_positions.items():
#     if v not in flipped:
#         flipped[v] = [k]
#     else:
#         flipped[v].append(k)
# [el for k,v in flipped.items() if len(v) > 1 for el in v]

In [10]:
t = np.array([[1,1,0,1],[0,0,1,0],[0,0,1,0]])
t

array([[1, 1, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [13]:
t[:,0][t[:,0]>0] = 5
t

array([[5, 1, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [30]:
(t[:,0:2] == [0,0]).all(-1).any(-1)

True

In [45]:
def print_array(np_blizzards, height, width):
    ret = ''
    for i in range(height):
        for j in range(width):
            if (i in (0,height-1)) or (j in (0, width-1)):
                ret += '#'
            else:
                if (np_blizzards[:,0:2] == [i,j]).all(-1).any(-1):
                    ret += '@'
                else:
                    ret += '.'
        ret += '\n'
    print(ret)

def move_blizzards(np_blizzards, height, width) -> None:
    ##move blizards
    np_blizzards[:,0:2] += np_blizzards[:,2:4]
    
    ##fix positions
    np_blizzards[:,0][np_blizzards[:,0] == 0] = height - 2
    np_blizzards[:,0][np_blizzards[:,0] == (height - 1)] = 1

    np_blizzards[:,1][np_blizzards[:,1] == 0] = width - 2
    np_blizzards[:,1][np_blizzards[:,1] == (width - 1)] = 1

# print_array(np_blizzards, height, width)

In [152]:
# We will store a dictionary with mapping {old_position: new_position, ...}
# This mapping is going to be the same for the initial data load but for each iteration we will update the dictionary values, test for collisions (by reversing the dict) and posibly undo the move for collisions

file1 = open('24.txt', 'r')
lines = file1.readlines()

blizzards = []
height = 0
width = 0

for i, l in enumerate(lines):
    height = i +1
    for j,el in enumerate(l.replace('\n','')):
        width = j + 1
        if el == '>':
            blizzards.append([i,j,0,1])
        elif el == '<':
            blizzards.append([i,j,0,-1])
        elif el == 'v':
            blizzards.append([i,j,1,0])
        elif el == '^':
            blizzards.append([i,j,-1,0])
np_blizzards = np.array(blizzards)
# print(np_blizzards)
print(height,width)

# print_array(np_blizzards, height, width)


27 122


In [62]:
# move_blizzards(np_blizzards, height, width)
# print_array(np_blizzards, height, width)

In [134]:
np_blizzards = np.array(blizzards)

## use BFS to find all positions
states = [[(0,1),[]]] # each state is (location),[historical locations]
steps = 0
min_distance = 999999
cnt = True
while cnt and states:
    new_states = []
    positions = []
    move_blizzards(np_blizzards, height, width)
    steps += 1
    for location, history in states:
        history = history.copy()
        history.append(location)
        li, lj = location
        
        for i,j in [(1,0),(-1,0),(0,1),(0,-1),(0,0)]:
            #check is blizard is not here
            if (np_blizzards[:,0:2] == [li+i,lj+j]).all(-1).any(-1) == False:
                if (li+i == height -1) and (lj+j == width -2):
                    cnt = False
                    print(f'final! {steps=}. {[li+i,lj+j]=}, {history=}')
                    break
                if ((li+i == 0) and (lj+j == 1)) or ((li+i > 0) and (li+i < height -1) and (lj+j > 0) and (lj+j < width -1)):
                    distance = abs((height -1)-(li+i)) + abs((width -2) - (lj+j))
                    if min_distance >= distance:
                        min_distance = distance
                    if (li+i,lj+j) not in positions: #distance >= min_distance-5 and 
                        new_states.append([(li+i,lj+j),history])
                        positions.append((li+i,lj+j))
                    # print((li+i,lj+j),'yes')
        if not cnt:
            break
    states = new_states
    print(f'{steps=}, states: {len(states)}, {min_distance=}')
    # break


steps=1, states: 1, min_distance=145
steps=2, states: 1, min_distance=145
steps=3, states: 2, min_distance=144
steps=4, states: 2, min_distance=143
steps=5, states: 3, min_distance=143
steps=6, states: 3, min_distance=143
steps=7, states: 5, min_distance=142
steps=8, states: 4, min_distance=142
steps=9, states: 5, min_distance=141
steps=10, states: 3, min_distance=140
steps=11, states: 6, min_distance=140
steps=12, states: 5, min_distance=140
steps=13, states: 8, min_distance=139
steps=14, states: 9, min_distance=138
steps=15, states: 12, min_distance=137
steps=16, states: 18, min_distance=136
steps=17, states: 18, min_distance=135
steps=18, states: 15, min_distance=135
steps=19, states: 16, min_distance=135
steps=20, states: 12, min_distance=135
steps=21, states: 10, min_distance=134
steps=22, states: 13, min_distance=133
steps=23, states: 15, min_distance=132
steps=24, states: 15, min_distance=131
steps=25, states: 15, min_distance=131
steps=26, states: 21, min_distance=131
steps=27,

In [ ]:
281

# Second part

In [153]:
np_blizzards = np.array(blizzards)
ans2 = []

## use BFS to find all positions
init_loc = (0,1)
final_loc = (height -1,width -2)
states = [[init_loc,[]]] # each state is (location),[historical locations]
steps = 0
min_distance = 999999
cnt = True

while cnt and states:
    new_states = []
    positions = []
    move_blizzards(np_blizzards, height, width)
    steps += 1
    for location, history in states:
        history = history.copy()
        history.append(location)
        li, lj = location
        
        for i,j in [(1,0),(-1,0),(0,1),(0,-1),(0,0)]:
            #check is blizard is not here
            if (np_blizzards[:,0:2] == [li+i,lj+j]).all(-1).any(-1) == False:
                if (li+i == final_loc[0]) and (lj+j == final_loc[1]):
                    cnt = False
                    print(f'final! {steps=}. {[li+i,lj+j]=}, {history=}')
                    ans2.append(steps)
                    break
                if ((li+i == init_loc[0]) and (lj+j == init_loc[1])) or ((li+i > 0) and (li+i < height -1) and (lj+j > 0) and (lj+j < width -1)):
                    distance = abs((final_loc[0])-(li+i)) + abs((final_loc[1]) - (lj+j))
                    if min_distance >= distance:
                        min_distance = distance
                    if (li+i,lj+j) not in positions: #distance >= min_distance-5 and 
                        new_states.append([(li+i,lj+j),history])
                        positions.append((li+i,lj+j))
                    # print((li+i,lj+j),'yes')
        if not cnt:
            break
    states = new_states
    print(f'{steps=}, states: {len(states)}, {min_distance=}')
    # break

init_loc = (height -1,width -2)
final_loc = (0,1)
states = [[init_loc,[]]] # each state is (location),[historical locations]
steps = 0
min_distance = 999999
cnt = True

while cnt and states:
    new_states = []
    positions = []
    move_blizzards(np_blizzards, height, width)
    steps += 1
    for location, history in states:
        history = history.copy()
        history.append(location)
        li, lj = location
        
        for i,j in [(1,0),(-1,0),(0,1),(0,-1),(0,0)]:
            #check is blizard is not here
            if (np_blizzards[:,0:2] == [li+i,lj+j]).all(-1).any(-1) == False:
                if (li+i == final_loc[0]) and (lj+j == final_loc[1]):
                    cnt = False
                    print(f'final! {steps=}. {[li+i,lj+j]=}, {history=}')
                    ans2.append(steps)
                    break
                if ((li+i == init_loc[0]) and (lj+j == init_loc[1])) or ((li+i > 0) and (li+i < height -1) and (lj+j > 0) and (lj+j < width -1)):
                    distance = abs((final_loc[0])-(li+i)) + abs((final_loc[1]) - (lj+j))
                    if min_distance >= distance:
                        min_distance = distance
                    if (li+i,lj+j) not in positions: #distance >= min_distance-5 and 
                        new_states.append([(li+i,lj+j),history])
                        positions.append((li+i,lj+j))
                    # print((li+i,lj+j),'yes')
                # else:
                    # print((li+i,lj+j),'no')
        if not cnt:
            print('break')
            break
    states = new_states
    print(f'{steps=}, states: {len(states)}, {min_distance=}')
    # break

init_loc = (0,1)
final_loc = (height -1,width -2)
states = [[init_loc,[]]] # each state is (location),[historical locations]
steps = 0
min_distance = 999999
cnt = True

while cnt and states:
    new_states = []
    positions = []
    move_blizzards(np_blizzards, height, width)
    steps += 1
    for location, history in states:
        history = history.copy()
        history.append(location)
        li, lj = location
        
        for i,j in [(1,0),(-1,0),(0,1),(0,-1),(0,0)]:
            #check is blizard is not here
            if (np_blizzards[:,0:2] == [li+i,lj+j]).all(-1).any(-1) == False:
                if (li+i == final_loc[0]) and (lj+j == final_loc[1]):
                    cnt = False
                    print(f'final! {steps=}. {[li+i,lj+j]=}, {history=}')
                    ans2.append(steps)
                    break
                if ((li+i == init_loc[0]) and (lj+j == init_loc[1])) or ((li+i > 0) and (li+i < height -1) and (lj+j > 0) and (lj+j < width -1)):
                    distance = abs((final_loc[0])-(li+i)) + abs((final_loc[1]) - (lj+j))
                    if min_distance >= distance:
                        min_distance = distance
                    if (li+i,lj+j) not in positions: #distance >= min_distance-5 and 
                        new_states.append([(li+i,lj+j),history])
                        positions.append((li+i,lj+j))
                    # print((li+i,lj+j),'yes')
        if not cnt:
            break
    states = new_states
    print(f'{steps=}, states: {len(states)}, {min_distance=}')
    # break


ans2

steps=1, states: 1, min_distance=145
steps=2, states: 1, min_distance=145
steps=3, states: 2, min_distance=144
steps=4, states: 2, min_distance=143
steps=5, states: 3, min_distance=143
steps=6, states: 3, min_distance=143
steps=7, states: 5, min_distance=142
steps=8, states: 4, min_distance=142
steps=9, states: 5, min_distance=141
steps=10, states: 3, min_distance=140
steps=11, states: 6, min_distance=140
steps=12, states: 5, min_distance=140
steps=13, states: 8, min_distance=139
steps=14, states: 9, min_distance=138
steps=15, states: 12, min_distance=137
steps=16, states: 18, min_distance=136
steps=17, states: 18, min_distance=135
steps=18, states: 15, min_distance=135
steps=19, states: 16, min_distance=135
steps=20, states: 12, min_distance=135
steps=21, states: 10, min_distance=134
steps=22, states: 13, min_distance=133
steps=23, states: 15, min_distance=132
steps=24, states: 15, min_distance=131
steps=25, states: 15, min_distance=131
steps=26, states: 21, min_distance=131
steps=27,

[281, 242, 284]

In [154]:
sum(ans2)

807